In [1]:
#import packages
import argparse
import math
import sys
import time
import copy
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('th')

#load data from csv files
data_train = np.genfromtxt('fashion-mnist_train.csv', delimiter = ",", skip_header = 1)
data_test = np.genfromtxt('fashion-mnist_test.csv', delimiter = ",", skip_header = 1)

#data extraction
X_train = data_train[:,1:]
y_train = data_train[:,0]
X_test = data_test[:,1:] = data_test[:,1:]

#data transformation
y_train = to_categorical(y_train) #convert to categories
num_classes = y_train.shape[1] #10 labels
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#split validation and train dataset randomly
np.random.seed(12345)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, random_state=12345)

#reshape data into images
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_val = X_val.reshape(X_val.shape[0], 1, img_rows, img_cols)

#setting parameters
batch_size = 128
epochs = 10
filter_pixel=5
noise = 1
droprate=0.2
input_shape = (1, img_rows, img_cols)

#setting architecture
#best and fastest architecture for this challenge that I have tried
cnn3 = Sequential([
    Conv2D(64, kernel_size=(filter_pixel, filter_pixel), strides=(1, 1), padding="same", activation='relu', 
           input_shape=input_shape),    
    MaxPooling2D(pool_size=(2, 2)), 
    Dropout(droprate),
    Flatten(),    
    Activation('relu'),    
    Dense(num_classes),
    Activation('softmax')
])

#compile the model
cnn3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#first train
his = cnn3.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(X_val, y_val), shuffle=True)

# acc
acc = his.history["acc"]
val_acc = his.history["val_acc"]

acc_List = []
valacc_List = []

for i in acc:
    acc_List.append(i*100)

for i in val_acc:
    valacc_List.append(i*100)

plt.grid(True)
plt.plot(acc_List)
plt.plot(valacc_List)
plt.xlabel('Orange_Line = val_acc          Blue_Line = acc', fontsize=12, color='red')
plt.title(r'convolution(MNIST Dataset) - Dropout 20%')
plt.show()

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
 1664/48000 [>.............................] - ETA: 108s - loss: 1.5854 - acc: 0.4495

KeyboardInterrupt: 